# Rockit

[Rockit](https://gitlab.mech.kuleuven.be/meco-software/rockit) (Rapid Optimal Control kit) is a software framework to quickly prototype optimal control problems (aka dynamic optimization) that may arise in engineering:
iterative learning (ILC), model predictive control (NMPC), motion planning.

Notably, the software allows free end-time problems and multi-stage optimal problems.
The software is currently focused on direct methods and relieas eavily on [CasADi](http://casadi.org).

## Hello World

(Taken from the [example directory](https://gitlab.mech.kuleuven.be/meco-software/rockit/blob/master/examples/hello_world.py))

To run, click in the _code cell_ below, and hit Shift+Enter. You may need to do 'Kernel->Restart' if you paused a while...

Import the project:

In [ ]:
from rockit import *

Start an optimal control environment with a time horizon of 10 seconds (free time problems can be configured with `FreeTime(initial_guess)`).

In [ ]:
ocp = Ocp(T=10)

Define two scalar states (vectors and matrices also supported):

In [1]:
x1 = ocp.state()
x2 = ocp.state()

NameError: name 'ocp' is not defined

Define one piece-wise constant control input (use `order=1` for piecewise linear):

In [ ]:
u = ocp.control()

Specify differential equations for states (time dependency supported with `ocp.t`, DAEs also supported with `ocp.algebraic` and `add_alg`):

In [ ]:
ocp.set_der(x1, (1 - x2**2) * x1 - x2 + u)
ocp.set_der(x2, x1)

Lagrange objective (Mayer term supported with `ocp.at_tf(expression)`):

In [ ]:
ocp.add_objective(ocp.integral(x1**2 + x2**2 + u**2))

Path constraints (must be valid on the whole time domain running from `t0` to `tf=t0+T`, grid options available):

In [ ]:
ocp.subject_to(x1 >= -0.25)
ocp.subject_to(-1 <= (u <= 1 ))

Boundary constraints:

In [ ]:
ocp.subject_to(ocp.at_t0(x1) == 0)
ocp.subject_to(ocp.at_t0(x2) == 1)

Pick an NLP solver backend (CasADi `nlpsol` plugin):

In [ ]:
ocp.solver('ipopt')

Pick a solution method:

In [5]:
method = MultipleShooting(N=10, M=1, intg='rk')
#method = DirectCollocation(N=20)
ocp.method(method)

NameError: name 'MultipleShooting' is not defined

Solve:

In [6]:
sol = ocp.solve()

NameError: name 'ocp' is not defined

Show structure:

In [ ]:
ocp.spy()

Post-processing:

In [ ]:
tsa, x1a = sol.sample(x1, grid='control')
tsa, x2a = sol.sample(x2, grid='control')

tsb, x1b = sol.sample(x1, grid='integrator')
tsb, x2b = sol.sample(x2, grid='integrator')


from pylab import *

figure(figsize=(10, 4))
subplot(1, 2, 1)
plot(tsb, x1b, '.-')
plot(tsa, x1a, 'o')
xlabel("Times [s]", fontsize=14)
grid(True)
title('State x1')

subplot(1, 2, 2)
plot(tsb, x2b, '.-')
plot(tsa, x2a, 'o')
legend(['grid_integrator', 'grid_control'])
xlabel("Times [s]", fontsize=14)
title('State x2')
grid(True)

tsol, usol = sol.sample(u, grid='integrator',refine=100)

figure()
plot(tsol,usol)
title("Control signal")
xlabel("Times [s]")
grid(True)

tsc, x1c = sol.sample(x1, grid='integrator', refine=100)

figure(figsize=(15, 4))
plot(tsc, x1c, '-')
plot(tsa, x1a, 'o')
plot(tsb, x1b, '.')
xlabel("Times [s]")
grid(True)